In [ ]:
# Cài đặt Tesseract OCR và ngôn ngữ Tiếng Việt (Vietnamese)
!sudo apt update
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-vie

# ❗ Bổ sung: Cài đặt OpenCV
!pip install pytesseract pillow opencv-python

# Import thư viện và Cấu hình Tesseract cho Colab
import pytesseract
from PIL import Image
import cv2 # Thêm thư viện OpenCV
from google.colab import files
import os
import numpy as np

# Cấu hình đường dẫn Tesseract cho Colab
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

print("Cài đặt môi trường hoàn tất.")

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
40 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

**DEMO BAN ĐẦU**

In [ ]:
# ----------------------------------------------------------------------
# KHÔNG CHẠY CODE NÀY NẾU BẠN CHƯA CHẠY Ô CODE 1 ĐÃ CẬP NHẬT TRÊN!
# ----------------------------------------------------------------------
# (Các phần import, FRAUD_KEYWORDS, và analyze_fraud vẫn giữ nguyên)

# 2.1. Hàm Tách Văn Bản Theo Vị Trí (Bổ sung)
def extract_text_by_side(image_path):
    """
    Tách ảnh thành hai nửa (trái: đối phương, phải: người dùng) và chạy OCR riêng biệt.
    Giả định: Tin nhắn LỪA ĐẢO thường nằm bên trái (người gửi).
    """
    try:
        # Đọc ảnh bằng OpenCV
        img_cv = cv2.imread(image_path)
        if img_cv is None:
            return None, None, f"LỖI: Không thể đọc file ảnh '{image_path}'."

        h, w, _ = img_cv.shape
        mid_point = w // 2

        # Cắt ảnh thành hai nửa
        # Cắt 50% bên trái (Giả định là tin nhắn của Đối phương)
        left_half = img_cv[:, :mid_point]
        # Cắt 50% bên phải (Giả định là tin nhắn của Tôi/Người dùng)
        right_half = img_cv[:, mid_point:]

        # Chuyển ảnh OpenCV sang PIL để chạy pytesseract
        left_pil = Image.fromarray(cv2.cvtColor(left_half, cv2.COLOR_BGR2RGB))
        right_pil = Image.fromarray(cv2.cvtColor(right_half, cv2.COLOR_BGR2RGB))

        # Chạy OCR
        text_left = pytesseract.image_to_string(left_pil, lang='vie').strip()
        text_right = pytesseract.image_to_string(right_pil, lang='vie').strip()

        return text_left, text_right, None

    except Exception as e:
        return None, None, f"Lỗi xử lý ảnh và OCR: {e}"

# 2.2. Hàm Phân tích Rủi ro đã được điều chỉnh
def analyze_fraud_on_opponent(opponent_text):
    """Đánh giá rủi ro chỉ dựa trên tin nhắn của đối phương."""
    # (Sử dụng lại hàm analyze_fraud nhưng chỉ với opponent_text)
    return analyze_fraud(opponent_text)


# 4. Thực thi: Tải file, Xử lý và In kết quả (Đã điều chỉnh)
print("Vui lòng chạy lệnh tiếp theo để mở hộp thoại chọn file ảnh...")

# 4.1. Tải file từ máy tính người dùng
uploaded = files.upload()

if not uploaded:
    print("❌ LỖI: Không có file ảnh nào được tải lên. Đã dừng xử lý.")
else:
    image_file = list(uploaded.keys())[0]
    print(f"\n✅ Đã tải file: {image_file}. Bắt đầu xử lý...")

    # ❗ Thực hiện tách văn bản theo bên
    text_opponent, text_user, error_message = extract_text_by_side(image_file)

    if error_message:
        print(f"\n❌ {error_message}")
    else:
        # Phân tích rủi ro chỉ trên tin nhắn của đối phương
        score, assessment, keywords = analyze_fraud_on_opponent(text_opponent)

        print("\n================ KẾT QUẢ PHÂN TÍCH ===============")
        print(f"1. FILE ẢNH ĐƯỢC XỬ LÝ: {image_file}")
        print("--------------------------------------------------")

        print(f"2. NỘI DUNG ĐƯỢC PHÂN BIỆT:")
        print("   -> Tin nhắn Đối phương (Bên Trái):")
        print(f"      {text_opponent[:100]}...")
        print("   -> Tin nhắn Người dùng (Bên Phải):")
        print(f"      {text_user[:100]}...")
        print("--------------------------------------------------")

        print(f"3. ĐÁNH GIÁ RỦI RO (Dựa trên tin nhắn Đối phương):")
        print(f"   -> Điểm rủi ro: {score}")
        print(f"   -> Kết luận: {assessment}")
        print("--------------------------------------------------")

        print(f"4. CÁC TỪ KHÓA CẢNH BÁO PHÁT HIỆN:")
        if keywords:
            print(f"   -> {', '.join(keywords)}")
        else:
            print("   -> Không tìm thấy từ khóa cảnh báo.")
        print("==================================================")

    # Xóa file đã tải lên
    try:
        os.remove(image_file)
        print(f"\nĐã xóa file tạm thời: {image_file}")
    except OSError:
        pass

Vui lòng chạy lệnh tiếp theo để mở hộp thoại chọn file ảnh...


Saving demo.jpg to demo (1).jpg

✅ Đã tải file: demo (1).jpg. Bắt đầu xử lý...

================ KẾT QUẢ PHÂN TÍCH ===============
1. FILE ẢNH ĐƯỢC XỬ LÝ: demo (1).jpg
--------------------------------------------------
2. NỘI DUNG ĐƯỢC PHÂN BIỆT:
   -> Tin nhắn Đối phương (Bên Trái):
      Em ơi, bé nhà chị đang thi cuộc thi ảnh, nhờ e bấm vào link n.
nhé

https://m.facebook.com/events/18...
   -> Tin nhắn Người dùng (Bên Phải):
      ày để bình chọn giúp c

đạ ok chị nha
@...
--------------------------------------------------
3. ĐÁNH GIÁ RỦI RO (Dựa trên tin nhắn Đối phương):
   -> Điểm rủi ro: 3
   -> Kết luận: RỦI RO THẤP: Nội dung trò chuyện có vẻ bình thường.
--------------------------------------------------
4. CÁC TỪ KHÓA CẢNH BÁO PHÁT HIỆN:
   -> bấm vào

Đã xóa file tạm thời: demo (1).jpg


**SAU KHI TỐI ƯU THÊM**

In [ ]:
import pytesseract
from PIL import Image
from google.colab import files
import os
import cv2 # Thư viện xử lý ảnh
import numpy as np # Hỗ trợ xử lý mảng

# ----------------------------------------------------------------------
# 1. BỘ TỪ KHÓA LỪA ĐẢO VÀ THANG ĐIỂM CẢNH BÁO MỚI
# ----------------------------------------------------------------------

# 1.1. Từ khóa Rủi ro CAO (Cộng 5 điểm): Yêu cầu tiền/thông tin nhạy cảm
HIGH_RISK_KEYWORDS = [
    "mã OTP", "mật khẩu", "chuyển tiền", "tài khoản ngân hàng",
    "thanh toán ngay", "số tài khoản", "nạp thẻ", "nạp tiền",
    "gửi gấp"
]

# 1.2. Từ khóa Rủi ro TRUNG BÌNH (Cộng 3 điểm): Yêu cầu hành động đáng ngờ / Click LINK
MEDIUM_RISK_KEYWORDS = [
    "vi phạm pháp luật", "cán bộ công an", "khóa tài khoản",
    "đăng nhập", "xác nhận", "tải về"
]

# 1.3. Từ khóa Rủi ro NGỮ CẢNH (Cộng 1 điểm): Thường xuất hiện trong các chiêu trò
CONTEXT_RISK_KEYWORDS = [
    "trúng thưởng", "nhận quà", "bình chọn", "cuộc thi ảnh", "trao giải",
    "ủng hộ", "voting", "làm gấp"
]

# 1.4. TỪ KHÓA ĐẶC BIỆT (BONUS SCORE) - Phân tích Link/Hành động
# Các từ khóa yêu cầu hành động trên link, sẽ được tính điểm Bonus.
LINK_ACTION_KEYWORDS = ["bấm vào link", "click vào link", "vào link này"]
BONUS_SCORE = 7 # Cộng thêm 7 điểm nếu phát hiện hành động click link

# ----------------------------------------------------------------------
# 2. CÁC HÀM XỬ LÝ
# ----------------------------------------------------------------------

# 2.1. Hàm Tách Văn Bản Theo Vị Trí (Bổ sung cho phân biệt người nói)
def extract_text_by_side(image_path):
    """Tách ảnh thành hai nửa (trái: đối phương, phải: người dùng) và chạy OCR."""
    try:
        img_cv = cv2.imread(image_path)
        if img_cv is None:
            return None, None, f"LỖI: Không thể đọc file ảnh '{image_path}'."

        h, w, _ = img_cv.shape
        # Cắt ảnh làm 2. Có thể điều chỉnh tỷ lệ nếu bố cục chat không phải 50/50
        mid_point = w // 2

        left_half = img_cv[:, :mid_point]
        right_half = img_cv[:, mid_point:]

        left_pil = Image.fromarray(cv2.cvtColor(left_half, cv2.COLOR_BGR2RGB))
        right_pil = Image.fromarray(cv2.cvtColor(right_half, cv2.COLOR_BGR2RGB))

        text_left = pytesseract.image_to_string(left_pil, lang='vie').strip()
        text_right = pytesseract.image_to_string(right_pil, lang='vie').strip()

        return text_left, text_right, None

    except Exception as e:
        return None, None, f"Lỗi xử lý ảnh và OCR: {e}"


# 2.2. Hàm Phân tích Rủi ro dựa trên Thang điểm MỚI
def analyze_fraud(text):
    """Đánh giá mức độ rủi ro lừa đảo dựa trên từ khóa theo 3 cấp độ + Bonus."""
    if not text or "LỖI:" in text:
        return 0, "Không có dữ liệu hợp lệ để phân tích.", set()

    risk_score = 0
    detected_keywords = set()
    lower_text = text.lower()

    # 3.1. Phân tích Rủi ro CAO (5 điểm)
    for keyword in HIGH_RISK_KEYWORDS:
        if keyword in lower_text:
            risk_score += 5
            detected_keywords.add(keyword)

    # 3.2. Phân tích Rủi ro TRUNG BÌNH (3 điểm)
    for keyword in MEDIUM_RISK_KEYWORDS:
        if keyword in lower_text:
            risk_score += 3
            detected_keywords.add(keyword)

    # 3.3. Phân tích Rủi ro NGỮ CẢNH (1 điểm)
    for keyword in CONTEXT_RISK_KEYWORDS:
        if keyword in lower_text:
            risk_score += 1
            detected_keywords.add(keyword)

    # 3.4. Bổ sung ĐIỂM BONUS: Nếu có yêu cầu click link
    for keyword in LINK_ACTION_KEYWORDS:
        if keyword in lower_text:
            risk_score += BONUS_SCORE # Cộng thêm 7 điểm
            detected_keywords.add(keyword)
            break # Chỉ cần tìm thấy một lần là đủ để cộng bonus

    return risk_score, detected_keywords


# ----------------------------------------------------------------------
# 4. THỰC THI: Tải file, Xử lý và In kết quả (Sử dụng hàm phân tích mới)
# ----------------------------------------------------------------------
print("Vui lòng chạy lệnh tiếp theo để mở hộp thoại chọn file ảnh...")

uploaded = files.upload()

if not uploaded:
    print("❌ LỖI: Không có file ảnh nào được tải lên. Đã dừng xử lý.")
else:
    image_file = list(uploaded.keys())[0]
    print(f"\n✅ Đã tải file: {image_file}. Bắt đầu xử lý...")

    # Thực hiện tách văn bản theo bên
    text_opponent, text_user, error_message = extract_text_by_side(image_file)

    if error_message:
        print(f"\n❌ {error_message}")
    else:
        # ❗ CHỈ PHÂN TÍCH TIN NHẮN CỦA ĐỐI PHƯƠNG
        score, keywords = analyze_fraud(text_opponent)

        # Định nghĩa lại kết luận dựa trên thang điểm mới
        if score >= 10:
            assessment = "RỦI RO CỰC KỲ CAO: CHẮC CHẮN là lừa đảo. Dừng mọi hành động ngay lập tức!"
        elif score >= 5:
            assessment = "RỦI RO CAO: Có khả năng lừa đảo mạnh mẽ. Cần kiểm tra nguồn tin gấp!"
        else:
            assessment = "RỦI RO THẤP: Nội dung trò chuyện có vẻ bình thường."


        print("\n================ KẾT QUẢ PHÂN TÍCH ===============")
        print(f"1. FILE ẢNH ĐƯỢC XỬ LÝ: {image_file}")
        print("--------------------------------------------------")

        print(f"2. NỘI DUNG ĐƯỢC PHÂN BIỆT:")
        print("   -> Tin nhắn Đối phương (Bên Trái - Được phân tích):")
        print(f"      {text_opponent[:100]}...")
        print("   -> Tin nhắn Người dùng (Bên Phải - Bỏ qua phân tích):")
        print(f"      {text_user[:100]}...")
        print("--------------------------------------------------")

        print(f"3. ĐÁNH GIÁ RỦI RO (Dựa trên tin nhắn Đối phương):")
        print(f"   -> Điểm rủi ro: {score}")
        print(f"   -> Kết luận: {assessment}")
        print("--------------------------------------------------")

        print(f"4. CÁC TỪ KHÓA CẢNH BÁO PHÁT HIỆN:")
        if keywords:
            # Gộp cả từ khóa hành động link (nếu có) vào danh sách in ra
            all_keywords = keywords.union(set(LINK_ACTION_KEYWORDS))
            print(f"   -> {', '.join(keywords)}")
        else:
            print("   -> Không tìm thấy từ khóa cảnh báo.")
        print("==================================================")

    # Xóa file đã tải lên
    try:
        os.remove(image_file)
        print(f"\nĐã xóa file tạm thời: {image_file}")
    except OSError:
        pass

Vui lòng chạy lệnh tiếp theo để mở hộp thoại chọn file ảnh...


Saving demo.jpg to demo (1).jpg

✅ Đã tải file: demo (1).jpg. Bắt đầu xử lý...

================ KẾT QUẢ PHÂN TÍCH ===============
1. FILE ẢNH ĐƯỢC XỬ LÝ: demo (1).jpg
--------------------------------------------------
2. NỘI DUNG ĐƯỢC PHÂN BIỆT:
   -> Tin nhắn Đối phương (Bên Trái - Được phân tích):
      Em ơi, bé nhà chị đang thi cuộc thi ảnh, nhờ e bấm vào link n.
nhé

https://m.facebook.com/events/18...
   -> Tin nhắn Người dùng (Bên Phải - Bỏ qua phân tích):
      ày để bình chọn giúp c

đạ ok chị nha
@...
--------------------------------------------------
3. ĐÁNH GIÁ RỦI RO (Dựa trên tin nhắn Đối phương):
   -> Điểm rủi ro: 8
   -> Kết luận: RỦI RO CAO: Có khả năng lừa đảo mạnh mẽ. Cần kiểm tra nguồn tin gấp!
--------------------------------------------------
4. CÁC TỪ KHÓA CẢNH BÁO PHÁT HIỆN:
   -> bấm vào link, cuộc thi ảnh

Đã xóa file tạm thời: demo (1).jpg


**DEMO VỚI ẢNH KHÔNG LỪA**

In [ ]:
import pytesseract
from PIL import Image
from google.colab import files
import os
import cv2 # Thư viện xử lý ảnh
import numpy as np # Hỗ trợ xử lý mảng

# ----------------------------------------------------------------------
# 1. BỘ TỪ KHÓA LỪA ĐẢO VÀ THANG ĐIỂM CẢNH BÁO MỚI
# ----------------------------------------------------------------------

# 1.1. Từ khóa Rủi ro CAO (Cộng 5 điểm): Yêu cầu tiền/thông tin nhạy cảm
HIGH_RISK_KEYWORDS = [
    "mã OTP", "mật khẩu", "chuyển tiền", "tài khoản ngân hàng",
    "thanh toán ngay", "số tài khoản", "nạp thẻ", "nạp tiền",
    "gửi gấp"
]

# 1.2. Từ khóa Rủi ro TRUNG BÌNH (Cộng 3 điểm): Yêu cầu hành động đáng ngờ / Click LINK
MEDIUM_RISK_KEYWORDS = [
    "vi phạm pháp luật", "cán bộ công an", "khóa tài khoản",
    "đăng nhập", "xác nhận", "tải về"
]

# 1.3. Từ khóa Rủi ro NGỮ CẢNH (Cộng 1 điểm): Thường xuất hiện trong các chiêu trò
CONTEXT_RISK_KEYWORDS = [
    "trúng thưởng", "nhận quà", "bình chọn", "cuộc thi ảnh", "trao giải",
    "ủng hộ", "voting", "làm gấp"
]

# 1.4. TỪ KHÓA ĐẶC BIỆT (BONUS SCORE) - Phân tích Link/Hành động
# Các từ khóa yêu cầu hành động trên link, sẽ được tính điểm Bonus.
LINK_ACTION_KEYWORDS = ["bấm vào link", "click vào link", "vào link này"]
BONUS_SCORE = 7 # Cộng thêm 7 điểm nếu phát hiện hành động click link

# ----------------------------------------------------------------------
# 2. CÁC HÀM XỬ LÝ
# ----------------------------------------------------------------------

# 2.1. Hàm Tách Văn Bản Theo Vị Trí (Bổ sung cho phân biệt người nói)
def extract_text_by_side(image_path):
    """Tách ảnh thành hai nửa (trái: đối phương, phải: người dùng) và chạy OCR."""
    try:
        img_cv = cv2.imread(image_path)
        if img_cv is None:
            return None, None, f"LỖI: Không thể đọc file ảnh '{image_path}'."

        h, w, _ = img_cv.shape
        # Cắt ảnh làm 2. Có thể điều chỉnh tỷ lệ nếu bố cục chat không phải 50/50
        mid_point = w // 2

        left_half = img_cv[:, :mid_point]
        right_half = img_cv[:, mid_point:]

        left_pil = Image.fromarray(cv2.cvtColor(left_half, cv2.COLOR_BGR2RGB))
        right_pil = Image.fromarray(cv2.cvtColor(right_half, cv2.COLOR_BGR2RGB))

        text_left = pytesseract.image_to_string(left_pil, lang='vie').strip()
        text_right = pytesseract.image_to_string(right_pil, lang='vie').strip()

        return text_left, text_right, None

    except Exception as e:
        return None, None, f"Lỗi xử lý ảnh và OCR: {e}"


# 2.2. Hàm Phân tích Rủi ro dựa trên Thang điểm MỚI
def analyze_fraud(text):
    """Đánh giá mức độ rủi ro lừa đảo dựa trên từ khóa theo 3 cấp độ + Bonus."""
    if not text or "LỖI:" in text:
        return 0, "Không có dữ liệu hợp lệ để phân tích.", set()

    risk_score = 0
    detected_keywords = set()
    lower_text = text.lower()

    # 3.1. Phân tích Rủi ro CAO (5 điểm)
    for keyword in HIGH_RISK_KEYWORDS:
        if keyword in lower_text:
            risk_score += 5
            detected_keywords.add(keyword)

    # 3.2. Phân tích Rủi ro TRUNG BÌNH (3 điểm)
    for keyword in MEDIUM_RISK_KEYWORDS:
        if keyword in lower_text:
            risk_score += 3
            detected_keywords.add(keyword)

    # 3.3. Phân tích Rủi ro NGỮ CẢNH (1 điểm)
    for keyword in CONTEXT_RISK_KEYWORDS:
        if keyword in lower_text:
            risk_score += 1
            detected_keywords.add(keyword)

    # 3.4. Bổ sung ĐIỂM BONUS: Nếu có yêu cầu click link
    for keyword in LINK_ACTION_KEYWORDS:
        if keyword in lower_text:
            risk_score += BONUS_SCORE # Cộng thêm 7 điểm
            detected_keywords.add(keyword)
            break # Chỉ cần tìm thấy một lần là đủ để cộng bonus

    return risk_score, detected_keywords


# ----------------------------------------------------------------------
# 4. THỰC THI: Tải file, Xử lý và In kết quả (Sử dụng hàm phân tích mới)
# ----------------------------------------------------------------------
print("Vui lòng chạy lệnh tiếp theo để mở hộp thoại chọn file ảnh...")

uploaded = files.upload()

if not uploaded:
    print("❌ LỖI: Không có file ảnh nào được tải lên. Đã dừng xử lý.")
else:
    image_file = list(uploaded.keys())[0]
    print(f"\n✅ Đã tải file: {image_file}. Bắt đầu xử lý...")

    # Thực hiện tách văn bản theo bên
    text_opponent, text_user, error_message = extract_text_by_side(image_file)

    if error_message:
        print(f"\n❌ {error_message}")
    else:
        # ❗ CHỈ PHÂN TÍCH TIN NHẮN CỦA ĐỐI PHƯƠNG
        score, keywords = analyze_fraud(text_opponent)

        # Định nghĩa lại kết luận dựa trên thang điểm mới
        if score >= 10:
            assessment = "RỦI RO CỰC KỲ CAO: CHẮC CHẮN là lừa đảo. Dừng mọi hành động ngay lập tức!"
        elif score >= 5:
            assessment = "RỦI RO CAO: Có khả năng lừa đảo mạnh mẽ. Cần kiểm tra nguồn tin gấp!"
        else:
            assessment = "RỦI RO THẤP: Nội dung trò chuyện có vẻ bình thường."


        print("\n================ KẾT QUẢ PHÂN TÍCH ===============")
        print(f"1. FILE ẢNH ĐƯỢC XỬ LÝ: {image_file}")
        print("--------------------------------------------------")

        print(f"2. NỘI DUNG ĐƯỢC PHÂN BIỆT:")
        print("   -> Tin nhắn Đối phương (Bên Trái - Được phân tích):")
        print(f"      {text_opponent[:100]}...")
        print("   -> Tin nhắn Người dùng (Bên Phải - Bỏ qua phân tích):")
        print(f"      {text_user[:100]}...")
        print("--------------------------------------------------")

        print(f"3. ĐÁNH GIÁ RỦI RO (Dựa trên tin nhắn Đối phương):")
        print(f"   -> Điểm rủi ro: {score}")
        print(f"   -> Kết luận: {assessment}")
        print("--------------------------------------------------")

        print(f"4. CÁC TỪ KHÓA CẢNH BÁO PHÁT HIỆN:")
        if keywords:
            # Gộp cả từ khóa hành động link (nếu có) vào danh sách in ra
            all_keywords = keywords.union(set(LINK_ACTION_KEYWORDS))
            print(f"   -> {', '.join(keywords)}")
        else:
            print("   -> Không tìm thấy từ khóa cảnh báo.")
        print("==================================================")

    # Xóa file đã tải lên
    try:
        os.remove(image_file)
        print(f"\nĐã xóa file tạm thời: {image_file}")
    except OSError:
        pass

Vui lòng chạy lệnh tiếp theo để mở hộp thoại chọn file ảnh...


Saving DEMO2.jpg to DEMO2.jpg

✅ Đã tải file: DEMO2.jpg. Bắt đầu xử lý...

================ KẾT QUẢ PHÂN TÍCH ===============
1. FILE ẢNH ĐƯỢC XỬ LÝ: DEMO2.jpg
--------------------------------------------------
2. NỘI DUNG ĐƯỢC PHÂN BIỆT:
   -> Tin nhắn Đối phương (Bên Trái - Được phân tích):
      Thi

thuar @ 4
G

ê nghe mike bão
coi tình hình rg đ
Đạt

bão deadline hả

chỉ chớ t thấy t3 I
nhóm ...
   -> Tin nhắn Người dùng (Bên Phải - Bỏ qua phân tích):
      ớp t1 cái
rồi đó

& Bạn đã trả lời Thĩ

ê nghe mike bão vô...
--------------------------------------------------
3. ĐÁNH GIÁ RỦI RO (Dựa trên tin nhắn Đối phương):
   -> Điểm rủi ro: 0
   -> Kết luận: RỦI RO THẤP: Nội dung trò chuyện có vẻ bình thường.
--------------------------------------------------
4. CÁC TỪ KHÓA CẢNH BÁO PHÁT HIỆN:
   -> Không tìm thấy từ khóa cảnh báo.

Đã xóa file tạm thời: DEMO2.jpg
